<a href="https://colab.research.google.com/github/feist000/Study/blob/master/Text_Mining/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive Bayes Practice 
## 직접 구현


### * 긍정(1), 부정(0)
### * 새로운 메일이 왔는데 'happy weekend가 포함되어 있을 경우, 이 메일은 긍정인가, 부정인가?

In [ ]:
import pandas as pd
import numpy as np
input_file = pd.read_csv("C:/Users/chae won/Desktop/Folder/Data/Text_Mining/naivebayes_example.csv") # 읽어오기 
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [ ]:
training_set = np.array(input_file)
print(training_set)

[['i love you' 1]
 ['love happy weekend' 1]
 ['bore work job' 0]
 ['i hate you' 0]
 ['bore weekend' 0]
 ['happy together' 1]]


### 토큰 빈도수 및 문서별 토큰 수 계산
#### 확률 계산을 위한 준비

In [ ]:
training_set

array([['i love you', 1],
       ['love happy weekend', 1],
       ['bore work job', 0],
       ['i hate you', 0],
       ['bore weekend', 0],
       ['happy together', 1]], dtype=object)

In [ ]:
# 토큰별로 문서 내 빈도수 카운팅
from collections import defaultdict
wordfreq = defaultdict(lambda : [0,0])  # lambda : 새로운 단어가 추가 될때 default 를 [0,0]으로 하겠다. [긍정,부정]
for doc, point in training_set:
    words = doc.split()
    for word in words:
        if point == 1:   
            wordfreq[word][0] +=1 # 긍정인 경우 0번째 인텍스에 1더해라 
        else:
            wordfreq[word][1] +=1 # 부정인 경우 1번째 인텍스에 1더해라
        
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [ ]:

# 긍정/ 부정 빈도수 계산
긍정전체토큰수=[]
부정전체토큰수=[]
for key, (cnt1,cnt0) in wordfreq.items():
    긍정전체토큰수.append(int(cnt1))
    부정전체토큰수.append(int(cnt0))
print(긍정전체토큰수)
print(부정전체토큰수)
전체갯수_긍정 = sum(긍정전체토큰수)
전체갯수_부정 = sum(부정전체토큰수)
print(전체갯수_긍정,전체갯수_부정)

[1, 2, 1, 2, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 2, 1, 1, 1, 0]
8 8


### Training : 토큰별 조건부 확률 계산

In [ ]:
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [ ]:
wordprobs = defaultdict(lambda:[0,0])
for key, (cnt1,cnt0) in wordfreq.items():
    wordprobs[key][0] = (cnt1 + 1)/(전체갯수_긍정 + len(wordfreq))
    wordprobs[key][1] = (cnt0 + 1)/(전체갯수_부정 + len(wordfreq))
wordprobs

defaultdict(<function __main__.<lambda>()>,
            {'i': [0.1111111111111111, 0.1111111111111111],
             'love': [0.16666666666666666, 0.05555555555555555],
             'you': [0.1111111111111111, 0.1111111111111111],
             'happy': [0.16666666666666666, 0.05555555555555555],
             'weekend': [0.1111111111111111, 0.1111111111111111],
             'bore': [0.05555555555555555, 0.16666666666666666],
             'work': [0.05555555555555555, 0.1111111111111111],
             'job': [0.05555555555555555, 0.1111111111111111],
             'hate': [0.05555555555555555, 0.1111111111111111],
             'together': [0.1111111111111111, 0.05555555555555555]})

### Classify : 신규 텍스트가 주어졌을 때 확률 계산

In [ ]:
import math
doc = "happy weekend" 
tokens = doc.split()
tokens

['happy', 'weekend']

In [ ]:

# 초기값을 모두 0으로 처리 
log_prob1 = log_prob0 = 0.0

# 모든 단어에 대해 반복 
for word, (prob1,prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 +=math.log(prob1)
        log_prob0 +=math.log(prob0)
log_prob1 += math.log(전체갯수_긍정/(전체갯수_긍정+전체갯수_부정))
log_prob0 += math.log(전체갯수_부정/(전체갯수_긍정+전체갯수_부정))

prob1 = math.exp(log_prob1) 
print('prob1',prob1)
prob0 = math.exp(log_prob0)   
print('prob0',prob0)
print('happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률')
print("긍정확률 : {}%".format(prob1/(prob1+prob0)*100))
print("부정확률 : {}%".format(prob0/(prob1+prob0)*100))

prob1 0.009259259259259257
prob0 0.0030864197530864183
happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률
긍정확률 : 75.00000000000001%
부정확률 : 24.999999999999996%


### Using Sklearn

In [ ]:
import pandas as pd
input_file = pd.read_csv("C:/Users/chae won/Desktop/Folder/Data/Text_Mining/naivebayes_example.csv") 
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [ ]:
X_train = list(input_file['mail'])
Y_train = list(input_file['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
# CountVectorizer 선언
count_vect = CountVectorizer()
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [ ]:
# 예측
print(clf.predict(count_vect.transform(["happy weekend"]))) 
# 확률 
print(clf.predict_proba(count_vect.transform(["happy weekend"])))

[1]
[[0.25 0.75]]


**-- END**